<img src="https://cdn-images-1.medium.com/max/1600/1*fIjRtO5P8zc3pjs0E5hYkw.png">

# Homework

## Building

The service is built inside of a docker container using Bazel.

Below are instructions for building the web service.

In [1]:
%%bash
docker build -t runtime-homework-r351574nc3 .

Sending build context to Docker daemon   89.6kB
Step 1/4 : FROM r351574nc3/bazel-onbuild:latest
# Executing 2 build triggers
 ---> Running in 373680381d4c
Starting local Bazel server and connecting to it...
.................................
Loading: 
Loading: 0 packages loaded
Loading: 0 packages loaded
Loading: 0 packages loaded
Analyzing: target //cmd:cmd (2 packages loaded)
Analyzing: target //cmd:cmd (2 packages loaded)
Analyzing: target //cmd:cmd (6 packages loaded)
Analyzing: target //cmd:cmd (12 packages loaded)
Analyzing: target //cmd:cmd (12 packages loaded)
Analyzing: target //cmd:cmd (13 packages loaded)
Analyzing: target //cmd:cmd (13 packages loaded)
Analyzing: target //cmd:cmd (13 packages loaded)
INFO: SHA256 (https://codeload.github.com/bazelbuild/buildtools/zip/799e530642bac55de7e76728fa0c3161484899f6) = ea23bbec9e86205b71ef647e1755ae0ec400aa76aeb5d13913d3fc3a37afbb5f
Analyzing: target //cmd:cmd (16 packages loaded)
INFO: SHA256 (https://codeload.github.com/golang/tool

## Usage

Instructions for running the web service locally

In [2]:
%%bash
docker run --rm -p 8080:8080 runtime-homework-r351574nc3

[GIN-debug] [WARNING] Creating an Engine instance with the Logger and Recovery middleware already attached.

[GIN-debug] [WARNING] Running in "debug" mode. Switch to "release" mode in production.
 - using env:  export GIN_MODE=release
 - using code: gin.SetMode(gin.ReleaseMode)

[GIN-debug] POST   /cases                    --> main.createRouter.func1 (3 handlers)
[GIN-debug] Listening and serving HTTP on :8080


## Results

See [Support Cases](./Support Cases/Index.ipynb) page for runtime results and testing

## Solution

Originally, I tried to solve the problem natively in javascript because the data was in javascript. My instinct was to manipulate with map reduce and see what came out. This is what I came up with

In [3]:
%%javascript
const support_cases = [
  {"case_id": 102, "state": {"from": null, "to": "open"}, "timestamp": "2017-01-09T03:00:00Z"},
  {"case_id": 100, "state": {"from": null, "to": "open"}, "timestamp": "2017-01-01T00:00:00Z"},
  {"case_id": 101, "state": {"from": null, "to": "open"}, "timestamp": "2017-01-05T10:00:00Z"},
  {"case_id": 100, "assignee": "Otter", "team": "Support", "timestamp": "2017-01-01T00:00:00Z"},
  {"case_id": 100, "assignee": "Rabbit", "team": "Runtime", "timestamp": "2017-01-02T15:00:00Z"},
  {"case_id": 102, "state": {"from": "open", "to": "pending"}, "timestamp": "2017-01-09T11:00:00Z"},
  {"case_id": 100, "state": {"from": "open", "to": "pending"}, "timestamp": "2017-01-02T16:00:00Z"},
  {"case_id": 100, "state": {"from": "pending", "to": "open"}, "timestamp": "2017-01-07T06:00:00Z"},
  {"case_id": 104, "state": {"from": null, "to": "open"}, "timestamp": "2017-01-13T06:00:00Z"},
  {"case_id": 100, "state": {"from": "open", "to": "closed"}, "timestamp": "2017-01-08T06:00:00Z"},
  {"case_id": 102, "assignee": "Rabbit", "team": "Runtime", "timestamp": "2017-01-10T06:00:00Z"},
  {"case_id": 101, "assignee": "Otter", "team": "Support", "timestamp": "2017-01-05T10:00:00Z"},
  {"case_id": 103, "state": {"from": null, "to": "open"}, "timestamp": "2017-01-11T19:00:00Z"},
  {"case_id": 102, "state": {"from": "pending", "to": "open"}, "timestamp": "2017-01-10T07:00:00Z"},
  {"case_id": 102, "state": {"from": "open", "to": "closed"}, "timestamp": "2017-01-21T11:00:00Z"},
  {"case_id": 103, "assignee": "Rabbit", "team": "Runtime", "timestamp": "2017-01-11T22:00:00Z"},
  {"case_id": 101, "state": {"from": "open", "to": "closed"}, "timestamp": "2017-01-05T23:00:00Z"},
  {"case_id": 103, "assignee": "Otter", "team": "Support", "timestamp": "2017-01-12T02:00:00Z"},
  {"case_id": 104, "assignee": "Otter", "team": "Support", "timestamp": "2017-01-13T13:00:00Z"},
  {"case_id": 103, "assignee": "Rabbit", "team": "Runtime", "timestamp": "2017-01-12T15:00:00Z"},
  {"case_id": 103, "state": {"from": "open", "to": "closed"}, "timestamp": "2017-01-16T19:00:00Z"},
  {"case_id": 104, "assignee": "Rabbit", "team": "Runtime", "timestamp": "2017-01-13T15:00:00Z"},
  {"case_id": 104, "assignee": "Badger", "team": "Runtime", "timestamp": "2017-01-15T08:00:00Z"},
  {"case_id": 104, "assignee": "Otter", "team": "Support", "timestamp": "2017-01-15T12:00:00Z"}
].sort((a, b) => {
    return new Date(a.timestamp) - new Date(b.timestamp)
})
console.log("Sorted data: ", support_cases)
function isOpening(state) {
    return state && state.to === "open"
}
function isUnopening(state) {
    return state && state.from === "open"
}
function isRuntime(case_id) {
    return last_team_index[case_id] && last_team_index[case_id] === "Runtime"
}
function isOpen(case_id) {
    return last_state_index[case_id] && last_state_index[case_id].name === "open"
}
let last_state_index = [
]
let last_team_index = []
let transitions = support_cases
    .filter((event) => isOpening(event.state) || isUnopening(event.state) || event.team)
    .map((event) => {
        let last_state = last_state_index[event.case_id] ? last_state_index[event.case_id] : {}
        // Current state matches filter and we're changing
        if (isRuntime(event.case_id) && isOpen(event.case_id)) {
            if (isUnopening(event.state) || (event.team && event.team !== "Runtime")) {
                return {
                    "case_id": event.case_id,
                    "ts_from": last_state.timestamp,
                    "ts_to": event.timestamp
                }
            }
            return null // state change isn't important
        }
        if (event.team) {
            last_team_index[event.case_id] = event.team
            // If the case is currently open, need to record this to the index
            if (isOpen(event.case_id) && isRuntime(event.case_id)) {
                last_state_index[event.case_id].timestamp = event.timestamp
            }
            
            return null
        }
        if (event.state) {
            if (isOpening(event.state)) {
                last_state_index[event.case_id] = { "name": event.state.to }
                if (isRuntime(event.case_id)) {
                    last_state_index[event.case_id].timestamp = event.timestamp
                }
            }
            return null
        }
        
        throw new Error("Unknown")
    })
    .filter((event) => event !== null)
console.log("transitions ", transitions)


The above became the logic for the service. Now I just needed to build a microservice around it.

> **UPDATE** I ended up making huge changes to the above. It was overly complex and restrictive. Further, the javascript did not port well.